## Google Drive 연동

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## JAVA 설치
- JVM 실행 위해서는 JAVA 설치 필수

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

 ## Spark 설치
 - 기존 : Web Link를 통해서 다운 후, 압축파일 불기
 - 오늘 : 구글 드라이브에서 파일 가져오기

In [3]:
%cd /content/drive/MyDrive/멀티캠퍼스/spark

/content/drive/MyDrive/멀티캠퍼스/spark


In [4]:
!pwd

/content/drive/MyDrive/멀티캠퍼스/spark


In [5]:
!ls

spark-3.1.1-bin-hadoop2.7.tgz


In [6]:
!cp -r spark-3.1.1-bin-hadoop2.7.tgz /content

In [7]:
%cd /content/

/content


In [8]:
!pwd

/content


In [9]:
!ls

drive  sample_data  spark-3.1.1-bin-hadoop2.7.tgz


In [10]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz > /dev/null

## 환경변수 설정
- 일반적으로 vi 편집기를 열어서 작업
- 구글코랩 : os 라이브러리 사용해서 환경변수 지정

In [11]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

## PySpark 설치

In [12]:
!pip install -q pyspark==3.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 16.7 MB/s eta 0:00:00


## Ngrok 설정
- 회원가입해서 Token을 받는다.
- 링크 : https://ngrok.com/

In [13]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null

In [14]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=60feba5bd5e92e46c428741580f499e0522d37d85ca8fe7e5e2220c6870493ed
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [24]:
!./ngrok authtoken 2X9PpF2DqMesfWaMzGuOeMfHNxC_2W95ibFm3gAcfZbtJ2DCa

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# 7551hxosxCMUhFXL2A18g_7WXBHeNkUZ1Sg9eSgM7Kd

## 테스트

In [25]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mulCamp28').config('spark.ui.port','4050').getOrCreate()
spark

In [26]:
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

In [27]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/usr/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


https://4c95-34-139-255-88.ngrok-free.app/jobs/

In [30]:
strings = spark.read.text("/content/spark-3.1.1-bin-hadoop2.7/README.md")
print(strings)

DataFrame[value: string]


In [31]:
print(type(strings))

<class 'pyspark.sql.dataframe.DataFrame'>


In [37]:
filtered = strings.filter(strings.value.contains("Spark"))
print(filtered)
print(type(filtered))

DataFrame[value: string]
<class 'pyspark.sql.dataframe.DataFrame'>


In [38]:
!ls

drive  ngrok-stable-linux-amd64.zip  spark-3.1.1-bin-hadoop2.7
ngrok  sample_data		     spark-3.1.1-bin-hadoop2.7.tgz


In [40]:
# !cat spark-3.1.1-bin-hadoop2.7/README.md

In [41]:
filtered.count()

19

## CSV파일 불러오기
- CSV 포맷으로 파일 읽어서 데이터프레임에 저장함
- 스키마 추론, 쉼표로 구분된 컬럼 이름이 제공되는 헤더가 있음 지정

교재

In [44]:
mnm_file = '/content/drive/MyDrive/멀티캠퍼스/spark/chapter02/mnm_dataset.csv'

mnm_df = spark.read.format('csv').option("header", "true").option("inferSchema", "true").load(mnm_file)
mnm_df.show(n=5, truncate=False) # show, pandas head()

+-----+------+-----+
|State|Color |Count|
+-----+------+-----+
|TX   |Red   |20   |
|NV   |Blue  |66   |
|CO   |Blue  |79   |
|OR   |Blue  |71   |
|WA   |Yellow|93   |
+-----+------+-----+
only showing top 5 rows



- GROUP BY State별, Color별 갯수를 요약해보자

In [48]:
count_mnm_df = mnm_df.select("State","Color","Count").groupBy("State","Color").sum("Count").orderBy("sum(Count)", ascending=False)
count_mnm_df.show(n = 10, truncate=False)

+-----+------+----------+
|State|Color |sum(Count)|
+-----+------+----------+
|CA   |Yellow|100956    |
|WA   |Green |96486     |
|CA   |Brown |95762     |
|TX   |Green |95753     |
|TX   |Red   |95404     |
|CO   |Yellow|95038     |
|NM   |Red   |94699     |
|OR   |Orange|94514     |
|WY   |Green |94339     |
|NV   |Orange|93929     |
+-----+------+----------+
only showing top 10 rows



In [49]:
count_mnm_df.count() # 행의 갯수

60

- CA만 확인하자

In [51]:
ca_count_mnm_df = mnm_df.select('*')\
  .where(mnm_df.State == 'CA')\
  .groupBy("State","Color")\
  .sum("Count")\
  .orderBy("sum(Count)", ascending=False)

ca_count_mnm_df.show()

+-----+------+----------+
|State| Color|sum(Count)|
+-----+------+----------+
|   CA|Yellow|    100956|
|   CA| Brown|     95762|
|   CA| Green|     93505|
|   CA|   Red|     91527|
|   CA|Orange|     90311|
|   CA|  Blue|     89123|
+-----+------+----------+



## Spark 종료

In [18]:
# spark.stop()